![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)
## Deploy Azure SQL Edge container instance on a remote machine with Docker
This notebook will use SSH to connect to the remote machine, use Docker to pull and run the Azure SQL Edge image and connect to it in Azure Data Studio


<span style="color:red"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### Install dependencies

In [ ]:
import pandas,sys,getpass,os,json,html,time
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')

def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{cmd}\n')
    print(f'Successfully executed: {cmd}')

cmd = 'pip install paramiko'
run_command()

### Setting variables

In [ ]:
env_var_flag = "AZDATA_NB_VAR_SA_PASSWORD" in os.environ
password_name = 'SQL Server sa account password'
if env_var_flag:
    container_name = os.environ["AZDATA_NB_VAR_DOCKER_CONTAINER_NAME"]
    sql_password = os.environ["AZDATA_NB_VAR_SA_PASSWORD"]
    sql_port = os.environ["AZDATA_NB_VAR_DOCKER_PORT"]
    docker_registry = os.environ["AZDATA_NB_VAR_DOCKER_REGISTRY"]
    docker_repository = os.environ["AZDATA_NB_VAR_DOCKER_REPOSITORY"]
    docker_imagetag = os.environ["AZDATA_NB_VAR_DOCKER_IMAGETAG"]
    docker_username = os.environ["AZDATA_NB_VAR_DOCKER_USERNAME"]
    docker_password = os.environ["AZDATA_NB_VAR_DOCKER_PASSWORD"]
    ssh_target = os.environ["AZDATA_NB_VAR_SSH_TARGET"]
    ssh_username = os.environ["AZDATA_NB_VAR_SSH_USERNAME"]
    ssh_password = os.environ["AZDATA_NB_VAR_SSH_PASSWORD"]
else:
    container_name = 'SQLEDGE-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
    docker_registry = 'mcr.microsoft.com'
    docker_repository = 'azure-sql-edge-developer'
    docker_imagetag = 'latest'
    docker_username = ''
    docker_password = ''
    sql_password = getpass.getpass(prompt = password_name)
    password_confirm = getpass.getpass(prompt = f'Confirm {password_name}')
    if sql_password != password_confirm:
        sys.exit(f'{password_name} does not match the confirmation password.')
    sql_port = input('SQL Server port, default value is 1433')
    if len(sql_port) == 0:
        sql_port = '1433'
    ssh_target = input('Remote machine name or address')
    ssh_username = input('Username for remote machine')
    ssh_password = input('Password for remote machine')

print(f'{password_name}: ******')
print(f'Remote machine: {ssh_target}')
print(f'Remote machine username: {ssh_username}')
print(f'Container name: {container_name}')
print(f'Port: {sql_port}')
print(f'Docker registry: {docker_registry}')
print(f'Docker repository: {docker_repository}')
print(f'Image tag: {docker_imagetag}')

### Log in to remote machine

In [ ]:
import paramiko
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(ssh_target,username=ssh_username, password=ssh_password)

def run_command_remote():
    stdin, stdout, stderr = client.exec_command(f'bash --login -c "{cmd}"')
    for line in stdout:
        print(line.strip('\n'))
    stderr = stderr.readlines()
    if len(stderr) != 0:
        for line in stderr:
            print(line.strip('\n'))
        sys.exit(f'Command execution failed')
    else:
        print(f'Command executed successfully')

### Check dependencies on remote machine

In [ ]:
cmd = 'docker version'
run_command_remote()


### List all the containers

In [ ]:
cmd = 'docker ps -a'
run_command_remote()

### Pull the container image

In [ ]:
if docker_username != '':
    cmd = f'docker login {docker_registry} -u {docker_username} -p {docker_password}'
    run_command_remote()
cmd = f'docker pull {docker_registry}/{docker_repository}:{docker_imagetag}'
run_command_remote()

### Start a new container

In [ ]:
cmd = f'docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD={sql_password}" -p {sql_port}:1433 --name {container_name} -d {docker_registry}/{docker_repository}:{docker_imagetag}'
run_command_remote()

### List all the containers

In [ ]:
cmd = f'docker ps -a'
run_command_remote()

### Connect to Azure SQL Edge instance in Azure Data Studio
It might take a couple minutes for the service to launch

In [ ]:
from IPython.display import *
connectionParameter = '{"serverName":"' + ssh_target + ',' + sql_port + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(sql_password) + '}'
display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to the Azure SQL Edge instance</font></a><br/>'))
display(HTML('<br/><span style="color:red"><font size="2">NOTE: The Azure SQL Edge instance password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))

### Stop and remove the container
Please copy the output of this cell into the next empty code cell and run the cell to stop and remove the container

In [ ]:
remove_container_command = f'cmd = f\'docker rm {container_name}\'\nrun_command_remote()'
stop_container_command = f'cmd = f\'docker stop {container_name}\'\nrun_command_remote()'
print(stop_container_command)
print(remove_container_command)